In [34]:
import pandas as pd
from random import shuffle
import re
import numpy as np

In [35]:
dataorg = pd.read_csv("hintest.csv")

In [36]:
pd.isnull(dataorg).sum()

marwadi    0
hindi      3
dtype: int64

In [37]:
dataorg = dataorg.dropna()

In [39]:
data = pd.DataFrame()
data["hindi_sentance"] = dataorg["hindi"]
data["marwadi_sentence"] = dataorg["marwadi"]
data.head(10)

,hindi_sentance,marwadi_sentence
0,अच्छा,आछौ
1,आम,आंबौ
2,आकाश,आब
3,बहुत दूर,फिर
4,बचाओ!,बचा
5,कूदो,कूद
6,नमस्कार।,राम राम सा
7,वाह-वाह!,वाह-वाह
8,समझे कि नहीं?,समझे मे आयो
9,मैं ठीक हूँ।,मे चॉको हु


In [71]:
data

,hindi_sentance,marwadi_sentence
0,START_ अच्छा _END,आछौ
1,START_ आम _END,आंबौ
2,START_ आकाश _END,आब
3,START_ बहुत दूर _END,फिर
4,START_ बचाओ _END,बचा
...,...,...
991,START_ तुम्हारी शादी कब हुई थी _END,थारी शादी हुगी
992,START_ आप वहाँ जाएँगे क्या _END,थे बटे जावो काईं
993,START_ हाँ मुझे यह बहुत पसंद है। _END,हा मन ए कल्डा चोखा लागे
994,START_ आपको लाईन में लगकर इंतेज़ार करना पड़ेगा...,थाने लाईन में लागर अढिकणो पङी


In [40]:
data["hindi_sentance"].apply(lambda x: x.lower())
data["marwadi_sentence"].apply(lambda x: x.lower())

0                                 आछौ
1                               आंबौ 
2                                 आब 
3                                 फिर
4                                 बचा
                    ...              
991                   थारी शादी हुगी?
992                 थे बटे जावो काईं?
993          हा मन ए कल्डा चोखा लागे.
994    थाने लाईन में लागर अढिकणो पङी.
995               तु मन पागल कर दियो.
Name: marwadi_sentence, Length: 993, dtype: object

In [41]:
data["hindi_sentance"].apply(lambda x: re.sub("''",'',x))
data["marwadi_sentence"].apply(lambda x: re.sub("''",'',x))

0                                 आछौ
1                               आंबौ 
2                                 आब 
3                                 फिर
4                                 बचा
                    ...              
991                   थारी शादी हुगी?
992                 थे बटे जावो काईं?
993          हा मन ए कल्डा चोखा लागे.
994    थाने लाईन में लागर अढिकणो पङी.
995               तु मन पागल कर दियो.
Name: marwadi_sentence, Length: 993, dtype: object

In [42]:
import string
exclude = set(string.punctuation) # Set of all special characters
# Remove all the special characters
data['hindi_sentance']=data['hindi_sentance'].apply(lambda x: ''.join(ch for ch in x if ch not in exclude))
data['marwadi_sentence']=data['marwadi_sentence'].apply(lambda x: ''.join(ch for ch in x if ch not in exclude))

In [43]:
# Remove all numbers from text
remove_digits = str.maketrans('', '', string.digits)
data['hindi_sentance']=data['hindi_sentance'].apply(lambda x: x.translate(remove_digits))
data['marwadi_sentence']=data['marwadi_sentence'].apply(lambda x: x.translate(remove_digits))

data['hindi_sentance'] = data['hindi_sentance'].apply(lambda x: re.sub("[२३०८१५७९४६]", "", x))
data['marwadi_sentence'] = data['marwadi_sentence'].apply(lambda x: re.sub("[२३०८१५७९४६]", "", x))

# Remove extra spaces
data['hindi_sentance']=data['hindi_sentance'].apply(lambda x: x.strip())
data['marwadi_sentence']=data['marwadi_sentence'].apply(lambda x: x.strip())
data['hindi_sentance']=data['hindi_sentance'].apply(lambda x: re.sub(" +", " ", x))
data['marwadi_sentence']=data['marwadi_sentence'].apply(lambda x: re.sub(" +", " ", x))

In [44]:
data['hindi_sentance'] = data['hindi_sentance'].apply(lambda x : 'START_ '+ x + ' _END')

In [45]:
### Get Hinid and Marwadi Vocabulary
all_hin_words=set()
for eng in data['hindi_sentance']:
    for word in eng.split():
        if word not in all_hin_words:
            all_hin_words.add(word)

all_mar_words=set()
for hind in data['marwadi_sentence']:
    for word in hind.split():
        if word not in all_mar_words:
            all_mar_words.add(word)

In [46]:
x_vocab = len(all_hin_words)

In [47]:
y_vocab = len(all_mar_words)

In [48]:
MAX_HIN_LEN = 0
for i in data["hindi_sentance"]:
    if len(i.split())>MAX_HIN_LEN:
        MAX_HIN_LEN = len(i.split())
MAX_HIN_LEN

14

In [49]:
MAX_MAR_LEN = 0
for i in data["marwadi_sentence"]:
    if len(i.split())>MAX_MAR_LEN:
        MAX_MAR_LEN = len(i.split())
MAX_MAR_LEN

10

In [50]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
x_tokenizer = Tokenizer(num_words=x_vocab)
x_tokenizer.fit_on_texts(data["hindi_sentance"].values)
X = x_tokenizer.texts_to_sequences(data["hindi_sentance"].values)
X = pad_sequences(X, maxlen=MAX_HIN_LEN, padding="post")
X.shape

(993, 14)

In [51]:
y_tokenizer = Tokenizer(num_words=y_vocab)
y_tokenizer.fit_on_texts(data["marwadi_sentence"].values)
Y = y_tokenizer.texts_to_sequences(data["marwadi_sentence"].values)
Y = pad_sequences(Y, maxlen=MAX_MAR_LEN, padding="post")
Y.shape

(993, 10)

In [52]:
from sklearn.model_selection import train_test_split
x_tr, x_val, y_tr, y_val = train_test_split(X, Y, train_size=0.1, random_state=42, shuffle=True)

In [53]:
from tensorflow.keras.layers import LSTM, Embedding, Dense, TimeDistributed
from tensorflow.keras import Input
from tensorflow.keras.models import Model

#making encoder model
embedded_dim = 100
latent_dim = 300

encoder_input = Input(shape=(MAX_HIN_LEN,))
encoder_emb_layer = Embedding(x_vocab, embedded_dim)
encoder_emb = encoder_emb_layer(encoder_input)
encoder_lstm = LSTM(latent_dim, recurrent_dropout=0.4, dropout=0.4, return_sequences=True, return_state=True)
encoder_output, state_h, state_c = encoder_lstm(encoder_emb)

#decoding layer
decoder_input = Input(shape=(None,))
decoder_emb_layer = Embedding(y_vocab, embedded_dim)
decoder_emb = decoder_emb_layer(decoder_input)
decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True)
decoder_output, decoder_state_h, decoder_state_c = decoder_lstm(decoder_emb, initial_state=[state_h, state_c])

decoder_dense = TimeDistributed(Dense(y_vocab, activation="softmax"))
decoder_output = decoder_dense(decoder_output)

model = Model(inputs=[encoder_input, decoder_input], outputs=decoder_output)

In [54]:
model.compile(loss="sparse_categorical_crossentropy", optimizer="rmsprop", metrics=["accuracy"])

In [55]:
model.fit([x_tr,y_tr[:,:-1]],y_tr.reshape(y_tr.shape[0],y_tr.shape[1],1)[:,1:], epochs=500, batch_size=256, validation_data=([x_val,y_val[:,:-1]],y_val.reshape(y_val.shape[0],y_val.shape[1],1)[:,1:]))

Epoch 1/500
1/1 [==============================] - 4s 4s/step - loss: 7.2345 - accuracy: 0.0000e+00 - val_loss: 7.1983 - val_accuracy: 0.6460
Epoch 2/500
1/1 [==============================] - 0s 424ms/step - loss: 7.1988 - accuracy: 0.6240 - val_loss: 7.1607 - val_accuracy: 0.6460
Epoch 3/500
1/1 [==============================] - 0s 433ms/step - loss: 7.1620 - accuracy: 0.6240 - val_loss: 7.1018 - val_accuracy: 0.6460
Epoch 4/500
1/1 [==============================] - 0s 407ms/step - loss: 7.1042 - accuracy: 0.6240 - val_loss: 6.9711 - val_accuracy: 0.6460
Epoch 5/500
1/1 [==============================] - 0s 438ms/step - loss: 6.9770 - accuracy: 0.6240 - val_loss: 6.5030 - val_accuracy: 0.6460
Epoch 6/500
1/1 [==============================] - 0s 422ms/step - loss: 6.5205 - accuracy: 0.6240 - val_loss: 4.6612 - val_accuracy: 0.6460
Epoch 7/500
1/1 [==============================] - 0s 391ms/step - loss: 4.7223 - accuracy: 0.6240 - val_loss: 2.9548 - val_accuracy: 0.6460
Epoch 8/500


In [64]:
#making inference model for prediction

#for encoder model
encoder_model = Model(inputs=[encoder_input], outputs=[encoder_output,state_h,state_c])

decoder_internal_state_h = Input(shape=(latent_dim,))
decoder_internal_state_c = Input(shape=(latent_dim,))
decoder_internal_states = [decoder_internal_state_h, decoder_internal_state_c]


dec_emb_2 = decoder_emb_layer(decoder_input)
decoder_output, state_h2, state_c2 = decoder_lstm(dec_emb_2, initial_state=[decoder_internal_state_h, decoder_internal_state_c])
decoder_output = decoder_dense(decoder_output)
#for decoder model
decoder_model = Model(inputs=[decoder_input,decoder_internal_states], outputs=[decoder_output,state_h2,state_c2])

In [65]:
reverse_target_word_index=y_tokenizer.index_word
reverse_source_word_index=x_tokenizer.index_word
target_word_index=y_tokenizer.word_index

In [66]:
def decode_sequence(input_seq):
    e_out, e_h, e_c = encoder_model.predict(input_seq)
    
    target_seq = np.zeros((1, 1))
    target_seq[0, 0] = 1  # Use the index 1 for the 'start' token directly
    
    stop_condition = False
    decoded_seq = ''
    
    while not stop_condition:
        output_tokens, state_h2, state_c2 = decoder_model.predict([target_seq, e_h, e_c])
        sampled_token_index = np.argmax(output_tokens)
        
        if sampled_token_index == 0:  # Break the loop if the sampled token index is 0 (padding token)
            break
        
        sampled_token = reverse_target_word_index[sampled_token_index]
        
        if sampled_token != 'end':
            decoded_seq += ' ' + sampled_token
        
        if sampled_token == 'end' or len(decoded_seq.split()) >= MAX_HIN_LEN:
            stop_condition = True
        
        target_seq = np.zeros((1, 1))
        target_seq[0, 0] = sampled_token_index
        
        e_h, e_c = state_h2, state_c2
    
    return decoded_seq


In [68]:
def seq2text(input_seq):
    newString=''
    for i in input_seq:
        if(i!=0):
            newString=newString+reverse_source_word_index[i]+' '
    return newString

In [69]:
def seq2mar(input_seq):
    newString=''
    for i in input_seq:
        if(i!=0):
            newString=newString+reverse_target_word_index[i]+' '
    return newString

In [72]:
for i in range(1,50):  # Adjust the range based on the number of examples you want to test
    print('Input Hindi sentence:', seq2text(x_tr[i]))
    print("Real Marwadi translation: ", seq2mar(y_tr[i]))
    print("Predicted summary:", decode_sequence(x_tr[i].reshape(1, MAX_HIN_LEN)))
    print("\n")


Input Hindi sentence: start मैं आसपड़ोस में रहता हूँ। end 
Real Marwadi translation:  मैं अट कने ही रु 
1/1 [==============================] - 0s 22ms/step
Predicted summary:  मन रो चोखो है


Input Hindi sentence: start वह बीमार नहीं हो सकता। end 
Real Marwadi translation:  बो बीमार कोनी हु सके 
1/1 [==============================] - 0s 19ms/step
Predicted summary:  करो बिंया है


Input Hindi sentence: start उसको पैसों की कमी थी। end 
Real Marwadi translation:  बिन रिप्या की कमी ही। 
1/1 [==============================] - 0s 32ms/step
Predicted summary:  रिप्या रो सोरो है


Input Hindi sentence: start मुझे पता है उसने क्यों किया था। end 
Real Marwadi translation:  मने धयन है बि काई करियो 
1/1 [==============================] - 0s 20ms/step
Predicted summary:  एक में हु है


Input Hindi sentence: start वह बस आता ही होगा। end 
Real Marwadi translation:  बो बस आतो ही हुगो 
1/1 [==============================] - 0s 20ms/step
Predicted summary:  एक में ही है


Input Hindi sentence: start मै